In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot
import math
import random as ran
from scipy.stats import truncnorm
import pandas as pd

/Users/yerin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#timeseries data
all_data=pd.read_csv('../data/Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

dic_data = {k: v for k, v in all_data.groupby('date')}

In [3]:
#risk:cov_mat
def risk(date_str):
    risk_data=pd.read_csv('../data/Riskmodels/cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    return risk_data,risk_sedol,risk_mat
risk_data,risk_sedol,risk_mat=risk(date_list[0])

In [4]:
dic_data[list(dic_data.keys())[0]]['index']=list(dic_data[list(dic_data.keys())[0]].index)

bw=list(dic_data[list(dic_data.keys())[0]]['bw'])
bw=np.array(bw)
bw=bw.reshape(1,len(bw))

beta=list(dic_data[list(dic_data.keys())[0]]['beta'])
beta=np.array(beta)

alpha=list(dic_data[list(dic_data.keys())[0]]['as'])
alpha=np.array(alpha)
alpha=alpha.reshape(1,len(bw[0]))

sector_set = set(list(dic_data[list(dic_data.keys())[0]]['sector']))
sector_list = list(dic_data[list(dic_data.keys())[0]]['sector'])

gbs=dic_data[list(dic_data.keys())[0]][['sector', 'index']].apply(tuple, axis=1)
group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

mq_list=list(dic_data[list(dic_data.keys())[0]]['mq'])
mq_set=set(list(dic_data[list(dic_data.keys())[0]]['mq']))

gbm=dic_data[list(dic_data.keys())[0]][['mq', 'index']].apply(tuple, axis=1)
group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]


In [5]:
###real data 임시생성
size=1
a=[0]*size
for i in range(size):
    r = [ran.random() for i in range(len(bw[0]))]
    s = sum(r)
    r = [ i/s for i in r ]
    a[i]=r
a=np.array(a)


In [46]:
total_epoch = 200
#batch_size = 5
learning_rate = 0.0002

n_hidden = 256
n_input = len(bw[0])
n_noise = 1

In [47]:
X = tf.placeholder(tf.float32)
Z = tf.placeholder(tf.float32, [None, n_noise])

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.5))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.1))
G_b2 = tf.Variable(tf.zeros([n_input]))
c_9=tf.Variable(0.)
c_6=tf.Variable(0.)
c_7=tf.Variable(0.)
c_5=tf.Variable(0.)
c_8=tf.Variable(0.)
c_10=tf.Variable(0.)
c_11=tf.Variable(0.)

In [48]:
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1) + G_b1)
    #(3)
    out = tf.nn.relu(
                    tf.matmul(hidden, G_W2) + G_b2)

    #(4) w_i의 총합이 1이 되도록! 
    output=tf.div(
    out, tf.reduce_sum(out)
    )

    return output


In [49]:
def discriminator(inputs):
#    input_ = tf.convert_to_tensor(inputs, dtype=tf.float32)
#     #(5)
#     bww = tf.convert_to_tensor(bw, dtype=tf.float32)
#     d = tf.subtract(inputs,bww)
#     a_d = tf.abs(d)
#     out1 = tf.subtract(tf.constant(0.05,shape=[1,len(bw)]),a_d)
    
#     #(8)
#     betaa = tf.convert_to_tensor(beta, dtype=tf.float32)
#     beta_wi = tf.multiply(inputs,betaa)
#     beta_wb = tf.multiply(betaa,bww)
#     dbeta = tf.subtract(beta_wi,beta_wb)
#     a_dbeta = tf.abs(dbeta)
#     out2 = tf.subtract(tf.constant(0.1,shape=[1,len(bw)]),a_dbeta)
    
#     #output = tf.add(tf.reduce_sum(out1),tf.reduce_sum(out2))
#     output = tf.reduce_sum(out1)

    #(2) Objective Function!
    bww = tf.convert_to_tensor(bw, dtype=tf.float32)
    d = tf.subtract(inputs,bww)

    omega=tf.convert_to_tensor(risk_mat, dtype=tf.float32)
    t_d=tf.transpose(d)

    m1=tf.matmul(d,omega)
    m11=tf.matmul(m1,t_d)
    
    alphaa=tf.convert_to_tensor(alpha, dtype=tf.float32)
    m2=tf.matmul(alphaa,t_d)

    output=(tf.subtract(m11,m2))
    
    return output


In [50]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [53]:
G = generator(Z)
loss_G = discriminator(G)

saver = tf.train.Saver()
G_var_list = [G_W1, G_b1, G_W2, G_b2,c_5,c_8,c_10,c_11,c_9,c_6,c_7]

train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G,
                                                       var_list=G_var_list)


In [54]:
def constraints(dfd):
    
    # (5) d_i 범위
    c_5=0
    for i in range(len(list(bw[0]))):
        if abs(bw[0][i]-dfd[0][i])>0.05:
            c_5=c_5+1
    
    # (6) sector별 구분
    c_6=0
    for i in range(len(list(sector_set))):
        aa=-0.1
        for j in group_by_sector[i]:
            aa=aa+abs(dfd[0][j]-bw[0][j])
                   
        if aa>0 :
            c_6=c_6+len(group_by_sector[i])
    
    # (7) MCAPQ별 구분  
    c_7=0
    for i in range(len(list(mq_set))):
        aa=-0.1
        for j in group_by_mq[i]:
            aa=aa+abs(dfd[0][j]-bw[0][j])
                   
        if aa>0 :
            c_7=c_7+len(group_by_mq[i])
                   
    # (8) beta곱
    c_8=0
    if sum(abs(bw[0]-sol[0])*beta)>=0.1:
        c_8=c_8+len(beta)

    # (9) 자산 갯수 중 60개만 선택해라 !
    c_9=-60
    for j in range(len(bw[0])):
        if dfd[0][j]>0:
            c_9=c_9+1
                   
    if c_9<0 : c_9=-c_9
    else: c_9=c_9
    
    # (10) Active Share
    c_10=0
    if 0.5*sum(abs(dfd[0][j]-bw[0])*bw[0])<0.6:
        c_10=c_10+len(beta)
        
    # (11) Tracking Error
    c_11=0
    aj=np.dot((bw-sol[0]),risk_mat)
    jj=np.dot(aj,(bw-sol[0]).T)
    
    if 0.0025 > jj or 0.01 < jj : 
        c_11=len(bw[0])
    
    return c_5,c_6,c_7,c_8,c_9,c_10,c_11


In [55]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

loss_val_G = 0
best_sol=1000
best_sol_=0
for epoch in range(total_epoch):
    
    noise = get_noise(1, n_noise)
    sol= sess.run(G, feed_dict={Z: noise})

    c_5,c_6,c_7,c_8,c_9,c_10,c_11=constraints(sol)
        
    loss_G = discriminator(G)+c_5+c_6+c_7+c_8+c_9+c_10+c_11
    _, loss_val_G = sess.run([train_G, loss_G],
                             feed_dict={Z: noise})
    
    print(c_5,c_6,c_7,c_8,c_9,c_10,c_11)

    
# Feasibility Check !
    if c_9 <= 10:
        print(epoch,c_9)



# Best Solution ---------------
#     if loss_val_G < best_sol :
#         best_sol=loss_val_G
#         best_sol_=sol[0]
    
    #save_path = saver.save(sess, "./model.ckpt")


    print(loss_val_G)

0 423 446 492 192 492 492
[[ 2537.00146484]]
0 423 446 492 175 492 492
[[ 2520.00146484]]
0 423 446 492 192 492 492
[[ 2537.0012207]]
0 423 446 492 192 492 492
[[ 2537.0012207]]
0 391 446 492 175 492 492
[[ 2488.00146484]]
0 423 446 492 192 492 492
[[ 2537.00097656]]
0 423 446 492 192 492 492
[[ 2537.00097656]]
0 423 446 492 192 492 492
[[ 2537.00097656]]
0 391 446 492 174 492 492
[[ 2487.00146484]]
0 391 446 492 174 492 492
[[ 2487.0012207]]
0 391 446 492 174 492 492
[[ 2487.0012207]]
0 391 446 492 174 492 492
[[ 2487.0012207]]
0 423 446 492 191 492 492
[[ 2536.00097656]]
0 391 446 492 175 492 492
[[ 2488.0012207]]
0 423 446 492 191 492 492
[[ 2536.00097656]]
0 391 446 492 175 492 492
[[ 2488.0012207]]
0 423 446 492 191 492 492
[[ 2536.00097656]]
0 391 446 492 174 492 492
[[ 2487.0012207]]
0 423 446 492 203 492 492
[[ 2548.00097656]]
0 423 446 492 194 492 492
[[ 2539.00097656]]
0 423 446 492 195 492 492
[[ 2540.00097656]]
0 391 446 492 174 492 492
[[ 2487.00097656]]
0 423 446 492 195 

0 353 446 492 205 492 492
[[ 2480.00048828]]
0 353 446 492 204 492 492
[[ 2479.00048828]]
0 335 375 492 213 492 492
[[ 2399.00048828]]
0 335 375 492 214 492 492
[[ 2400.00048828]]
0 335 375 492 208 492 492
[[ 2394.00048828]]
0 353 446 492 205 492 492
[[ 2480.00048828]]
0 353 446 492 199 492 492
[[ 2474.00048828]]
0 335 375 492 216 492 492
[[ 2402.00048828]]
0 353 446 492 205 492 492
[[ 2480.00048828]]
0 335 375 492 217 492 492
[[ 2403.00048828]]
0 335 375 492 217 492 492
[[ 2403.00048828]]
0 353 446 492 205 492 492
[[ 2480.00048828]]
0 353 446 492 205 492 492
[[ 2480.00048828]]
0 353 446 492 205 492 492
[[ 2480.00048828]]
0 335 375 492 219 492 492
[[ 2405.00048828]]
0 335 375 492 217 492 492
[[ 2403.00048828]]
0 297 375 492 217 492 492
[[ 2365.00048828]]


In [56]:
sol

array([[ 0.        ,  0.        ,  0.00208089,  0.        ,  0.00397986,
         0.        ,  0.        ,  0.        ,  0.00682529,  0.00265759,
         0.00043596,  0.00332137,  0.00085012,  0.00686053,  0.        ,
         0.00134981,  0.00086419,  0.        ,  0.00167811,  0.        ,
         0.00500785,  0.        ,  0.00089118,  0.        ,  0.        ,
         0.00180107,  0.        ,  0.        ,  0.        ,  0.        ,
         0.00094581,  0.        ,  0.00237747,  0.        ,  0.00968861,
         0.00795276,  0.        ,  0.        ,  0.0008178 ,  0.        ,
         0.00029198,  0.        ,  0.00239776,  0.        ,  0.00076013,
         0.00295216,  0.00188314,  0.        ,  0.        ,  0.00123327,
         0.00112755,  0.00242949,  0.00054262,  0.00277486,  0.        ,
         0.00165547,  0.00161201,  0.        ,  0.00918804,  0.        ,
         0.00230018,  0.00152997,  0.00180998,  0.        ,  0.00185637,
         0.        ,  0.        ,  0.00061163,  0.0